### CONTENTS
#### 1 Importing Libraries
#### 2 Importing Data 
#### 3 Reshape Data for Deep Learning
#### 4 Baysian Optimization
#### 5 Fitting CNN Model with Optimal Parameters

# 1 Importing Libraries

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [17]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from bayes_opt import BayesianOptimization
from scikeras.wrappers import KerasClassifier
import tensorflow as tf

# 2 Importing Data

In [19]:
path = r'C:\Users\elena\Documents\06.2025 climatewins machine learning'

In [20]:
# Import the pleasant weather data.
pleasantweather = pd.read_csv(os.path.join(path, '02 Data', 'Original Data','Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [21]:
pleasantweather.drop(['DATE'], axis=1, inplace=True)

In [22]:
# Creat an 'X' matrix by reloading and naming our data 'X'
X=pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'weather_cleaned1.csv'), index_col=False)

In [23]:
y = pleasantweather

In [24]:
X.shape

(22950, 135)

In [33]:
y.shape

(22950, 15)

In [35]:
X.head()

BASEL_cloud_cover  BASEL_humidity  BASEL_pressure  BASEL_global_radiation  \
0                  7            0.85           1.018                    0.32   
1                  6            0.84           1.018                    0.36   
2                  8            0.90           1.018                    0.18   
3                  3            0.92           1.018                    0.58   
4                  6            0.95           1.018                    0.65   

   BASEL_precipitation  BASEL_sunshine  BASEL_temp_mean  BASEL_temp_min  \
0                 0.09             0.7              6.5             0.8   
1                 1.05             1.1              6.1             3.3   
2                 0.30             0.0              8.5             5.1   
3                 0.00             4.1              6.3             3.8   
4                 0.14             5.4              3.0            -0.7   

   BASEL_temp_max  BELGRADE_cloud_cover  BELGRADE_humidity  BELGRADE_pressure  \
0            10.9                     1               0.81             1.0195   
1            10.1                     6               0.84             1.0172   
2             9.9                     6               0.77             1.0179   
3            10.6                     8               0.93             1.0268   
4             6.0                     8               0.99             1.0286   

   BELGRADE_global_radiation  BELGRADE_precipitation  BELGRADE_sunshine  \
0                       0.88                    0.00                7.0   
1                       0.25                    0.00                0.0   
2                       0.67                    0.00                3.5   
3                       0.25                    0.00                0.0   
4                       0.25                    0.06                0.0   

   BELGRADE_temp_mean  BELGRADE_temp_min  BELGRADE_temp_max  \
0                 3.7               -0.9                7.9   
1                 2.9                2.2                4.4   
2                 3.1               -0.5                6.4   
3                 2.0               -2.0                3.0   
4                 2.0                0.7                2.8   

   BUDAPEST_cloud_cover  BUDAPEST_humidity  BUDAPEST_pressure  \
0                     4               0.67              1.017   
1                     4               0.67              1.017   
2                     4               0.67              1.017   
3                     4               0.67              1.017   
4                     4               0.67              1.017   

   BUDAPEST_global_radiation  BUDAPEST_precipitation  BUDAPEST_sunshine  \
0                       0.44                    0.01                2.3   
1                       0.18                    0.31                0.0   
2                       0.30                    0.00                0.6   
3                       0.19                    0.00                0.0   
4                       0.19                    0.00                0.0   

   BUDAPEST_temp_mean  BUDAPEST_temp_min  BUDAPEST_temp_max  \
0                 2.4               -0.4                5.1   
1                 2.3                1.4                3.1   
2                 2.7                1.7                5.3   
3                 2.0                0.4                4.4   
4                 2.5                1.1                5.3   

   DEBILT_cloud_cover  DEBILT_humidity  DEBILT_pressure  \
0                   7             0.85           1.0032   
1                   8             0.90           1.0056   
2                   6             0.92           1.0165   
3                   8             0.95           1.0265   
4                   6             0.90           1.0243   

   DEBILT_global_radiation  DEBILT_precipitation  DEBILT_sunshine  \
0                     0.07                  0.25              0.0   
1                     0.14              

# 3 Reshape Data for Deep Learning

In [38]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)

In [40]:

X = X.reshape(-1,15,9)

In [42]:
# Verify Shape
X.shape

(22950, 15, 9)

In [44]:
# Verify Shape
y.shape

(22950, 15)

In [46]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [48]:

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


# 4 Baysian Optimization

In [51]:
# Dimensions

timesteps = X_train.shape[1]  # 15
input_dim = X_train.shape[2]  # 9
n_classes = y_train.shape[1]  # 15

In [53]:
# Map optimizer names to classes
optimizer_classes = {
    'SGD': SGD,
    'Adam': Adam,
    'RMSprop': RMSprop,
    'Adagrad': Adagrad
}
activation_list = ['relu', 'sigmoid', 'tanh', 'selu', 'elu']

In [55]:
# Optimization function
def bayesian_cnn(neurons, kernel, activation, optimizer, learning_rate, batch_size, epochs,
                 layers1, layers2, normalization, dropout, dropout_rate):
    neurons = int(neurons)
    kernel = int(kernel)
    activation_fn = activation_list[int(activation)]
    optimizer_name = list(optimizer_classes.keys())[int(optimizer)]
    optimizer_instance = optimizer_classes[optimizer_name](learning_rate=learning_rate)
    batch_size = int(batch_size)
    epochs = int(epochs)
    layers1 = int(layers1)
    layers2 = int(layers2)

    def build_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel, activation=activation_fn, input_shape=(timesteps, input_dim)))
        if normalization > 0.5:
            model.add(BatchNormalization())
        for _ in range(layers1):
            model.add(Dense(neurons, activation=activation_fn))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate))
        for _ in range(layers2):
            model.add(Dense(neurons, activation=activation_fn))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer=optimizer_instance, metrics=[])
        return model

    kfold = KFold(n_splits=3, shuffle=True, random_state=42)
    f1_scores = []

    for train_idx, val_idx in kfold.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model = build_model()
        es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)
        model.fit(X_tr, y_tr, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])
        
        y_pred = model.predict(X_val)
        y_pred_binary = (y_pred > 0.5).astype(int)
        score = f1_score(y_val, y_pred_binary, average='weighted')
        f1_scores.append(score)
        score = f1_score(y_val, y_pred_binary, average='weighted')

        return float(score)  # Ensure it’s a native Python float

In [57]:
# Bounds for parameters
pbounds = {
    'neurons': (32, 128),
    'kernel': (1, 3),
    'activation': (0, len(activation_list) - 1),
    'optimizer': (0, len(optimizer_classes) - 1),
    'learning_rate': (0.001, 0.05),
    'batch_size': (64, 256),
    'epochs': (10, 50),
    'layers1': (1, 3),
    'layers2': (1, 3),
    'normalization': (0, 1),
    'dropout': (0, 1),
    'dropout_rate': (0.0, 0.3)
}


In [331]:
# Run Bayesian Optimization
start = time.time()
optimizer = BayesianOptimization(f=bayesian_cnn, pbounds=pbounds, random_state=42, verbose=2)
optimizer.maximize(init_points=5, n_iter=10)  # You can increase these for deeper search
print(f"Search took {(time.time() - start) / 60:.2f} minutes")

|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  
| 1         | 0.0293308 | 67.955851 | 2.9014286 | 2.9279757 | 1.7959754 | 0.0086449 | 93.950947 | 12.323344 | 2.7323522 | 2.2022300 | 0.7080725 | 0.0205844 | 0.2909729 |
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  
| 2         | 0.7553262 | 111.91449 | 1.4246782 | 0.7272998 | 0.5502135 | 0.0159078 | 164.75323 | 27.277800 | 1.5824582 | 2.2237057 | 0.1394938 | 0.2921446 | 0.1099085 |
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  
| 3         | 0.8347941 | 75.782718 | 2.5703519 | 0.7986951 | 1.5427033 | 0.0300283 | 72.918479 | 34.301794 | 1.3410482 | 1.1301031 | 0.9488855 | 0.9656320 | 0.2425192 |
180/180 ━━━━━━━━━━━━━

In [333]:
# View best result
print("Best parameters found:")
print(optimizer.max)

Best parameters found:
{'target': 0.8758437466197764, 'params': {'neurons': 75.19867280264145, 'kernel': 2.3594728898566135, 'activation': 0.5012801619539795, 'optimizer': 1.6183164990712058, 'learning_rate': 0.03050203340104459, 'batch_size': 72.87929317614811, 'epochs': 33.551920394937916, 'layers1': 1.4438308309896044, 'layers2': 1.706588531074978, 'normalization': 1.0, 'dropout': 0.8151286545481646, 'dropout_rate': 0.21959164517513607}}


# 5 Fitting CNN Model with Optimal Parameters

In [59]:
activation_list = ['relu', 'tanh', 'sigmoid', 'elu', 'selu', 'gelu']  # index 5 ≈ 'gelu' → changed to 'relu' if unsupported
optimizer_list = ['adam', 'rmsprop', 'sgd', 'adagrad', 'nadam', 'adamax']  # index 5 ≈ 'adamax'


In [61]:
# --- Use optimized parameters ---
params = {
    'neurons': int(round(33.23695423568694)),
    'kernel': int(round(4.446384285364502)),
    'activation': 'relu',  # 5.65 maps to relu in your optimization
    'optimizer': 'adam',   # 5.10 maps to adam
    'learning_rate': 0.007735576432190864,
    'batch_size': int(round(68.73885771098179)),
    'epochs': int(round(55.09260099809909)),
    'layers1': int(round(1.4634762381005189)),
    'layers2': int(round(4.4524137035023745)),
    'normalization': True if 0.6232981268275579 > 0.5 else False,
    'dropout': True if 0.3308980248526492 > 0.5 else False,
    'dropout_rate': 0.12542334011440948
}

In [63]:
# --- Build model ---
input_shape = (X_train.shape[1], X_train.shape[2])  # (15, 9)
n_classes = y_train.shape[1]  # 15

model = Sequential()


In [65]:
# Input Conv1D layer
model.add(Conv1D(filters=params['neurons'],
                 kernel_size=params['kernel'],
                 activation=params['activation'],
                 input_shape=input_shape))

In [67]:
# Optional normalization
if params['normalization']:
    model.add(BatchNormalization())


In [69]:
# Optional dropout
if params['dropout']:
    model.add(Dropout(params['dropout_rate']))

In [71]:
# Add intermediate Conv1D layers (layers2 - 1 more)
for _ in range(max(params['layers2'] - 1, 0)):
    model.add(Conv1D(filters=params['neurons'],
                     kernel_size=params['kernel'],
                     activation=params['activation']))
    if params['normalization']:
        model.add(BatchNormalization())
    if params['dropout']:
        model.add(Dropout(params['dropout_rate']))


In [73]:
# Optional MaxPooling
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

In [75]:
# Dense layers
for _ in range(params['layers1']):
    model.add(Dense(params['neurons'], activation=params['activation']))
    if params['dropout']:
        model.add(Dropout(params['dropout_rate']))

In [77]:
# Output layer
model.add(Dense(n_classes, activation='sigmoid'))  # multi-label classification

In [79]:
# Optimizer selection
opt_dict = {
    'adam': Adam(learning_rate=params['learning_rate']),
    'rmsprop': RMSprop(learning_rate=params['learning_rate']),
    'sgd': SGD(learning_rate=params['learning_rate']),
    'adagrad': Adagrad(learning_rate=params['learning_rate']),
    'adadelta': Adadelta(learning_rate=params['learning_rate'])
}
optimizer = opt_dict.get(params['optimizer'], Adam(learning_rate=params['learning_rate']))


In [81]:
# Compile model
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [83]:
# Fit model
es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=1)
history = model.fit(X_train, y_train,
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    verbose=2,
                    callbacks=[es])

Epoch 1/55
250/250 - 7s - 30ms/step - accuracy: 0.1498 - loss: 0.2488
Epoch 2/55
250/250 - 2s - 8ms/step - accuracy: 0.1902 - loss: 0.1888
Epoch 3/55
250/250 - 2s - 8ms/step - accuracy: 0.2099 - loss: 0.1684
Epoch 4/55
250/250 - 2s - 8ms/step - accuracy: 0.2163 - loss: 0.1514
Epoch 5/55
250/250 - 2s - 8ms/step - accuracy: 0.2129 - loss: 0.1394
Epoch 6/55
250/250 - 2s - 7ms/step - accuracy: 0.2186 - loss: 0.1254
Epoch 7/55
250/250 - 2s - 7ms/step - accuracy: 0.2303 - loss: 0.1167
Epoch 8/55
250/250 - 2s - 8ms/step - accuracy: 0.2247 - loss: 0.1109
Epoch 9/55
250/250 - 2s - 8ms/step - accuracy: 0.2189 - loss: 0.1023
Epoch 10/55
250/250 - 2s - 8ms/step - accuracy: 0.2228 - loss: 0.0967
Epoch 11/55
250/250 - 2s - 8ms/step - accuracy: 0.2397 - loss: 0.0893
Epoch 12/55
250/250 - 2s - 8ms/step - accuracy: 0.2233 - loss: 0.0851
Epoch 13/55
250/250 - 2s - 8ms/step - accuracy: 0.2413 - loss: 0.0842
Epoch 14/55
250/250 - 2s - 8ms/step - accuracy: 0.2212 - loss: 0.0835
Epoch 15/55
250/250 - 2s - 7

In [87]:
from sklearn.metrics import classification_report

In [89]:
# Predict and report
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
print(classification_report(y_test, y_pred, zero_division=0))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1400
           1       0.99      0.99      0.99      1962
           2       0.99      0.99      0.99      1838
           3       0.99      0.97      0.98      1101
           4       0.99      0.96      0.97      1231
           5       0.97      0.98      0.98      1168
           6       0.99      0.97      0.98       923
           7       0.99      0.98      0.99      1543
           8       0.99      0.94      0.97      1176
           9       0.99      0.99      0.99      2570
          10       0.98      0.97      0.98      1192
          11       0.95      0.68      0.79       859
          12       0.00      0.00      0.00         0
          13       0.96      0.70      0.81       972
          14       0.40      0.03      0.05       276

   micro avg       0.98      0.94      0.96     18211
   macro avg       0.88      0.81     

Good model performance with the optimized parameters. 

In [92]:
# Confusion Matrix

In [94]:
from sklearn.metrics import multilabel_confusion_matrix

In [100]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [102]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [104]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         468       209       404      31         231       318      63   
BELGRADE        0       454       345      21          76        27       4   
BUDAPEST        0         0       151       1           9         6       0   
DEBILT          0         0         0      12          20        23       0   
DUSSELDORF      0         0         0       0           4         8       0   
HEATHROW        0         0         0       0           0        29       0   
KASSEL          0         0         0       0           0         0       5   
LJUBLJANA       0         0         0       0           0         0       0   
MAASTRICHT      0         0         0       0           0         0       0   
MADRID          0         0         0       0           0         0       0   
MUNCHENB  